In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from io import BytesIO
import cv2
import bson
from skimage.data import imread
import matplotlib.pyplot as plt
import keras
from PIL import Image
import io, bson, multiprocessing
import os
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

In [3]:
def get_the_data(path):
    data = bson.decode_file_iter(open(path, 'rb'))
    images=[]
    category=[]
    for c, d in enumerate(data):
        product_id = d['_id']
        category_id = d['category_id'] 
        for e, pic in enumerate(d['imgs']):
            category.append(category_id)
            im=pic['picture']
            images.append(Image.open(io.BytesIO(im)))
        if(len(set(category))==860):
            break
    return category, images

In [4]:
product_category_train,image_train=get_the_data('E:\\kaggle\\train.bson')

In [5]:
y, rev_labels = pd.factorize(product_category_train)

In [6]:
len(y)

9998

In [7]:
from sklearn.utils import shuffle
im_train,lab_train=shuffle(image_train,y)
test_im=im_train[9980:]
test_lab=lab_train[9980:]
image_train=im_train[:9980]
label_train=lab_train[:9980]

In [8]:
len_train = len(image_train)
len_test = len(test_im)

In [ ]:
len_cat = len(image_train)
#
cnt = 0
flag=0
for i in range(len_cat):
        if i == 0:
            n = label_train[i]
            #print('cat', n)
            dir_ = 'C:\\Users\\saish\\Documents\\EE258\\test\\train'
            pathname = os.path.join(dir_, str(n))
            if not os.path.exists(pathname):
                  os.makedirs(pathname)
            os.chdir(pathname)
            d = image_train[i]
            d.save("{0}.jpg".format(str(cnt)))
        if i != 0:      
            if((label_train[i])!=(label_train[i-1])):
                n = label_train[i]
                #print('cat', n)
                dir_ = 'C:\\Users\\saish\\Documents\\EE258\\test\\train'
                pathname = os.path.join(dir_, str(n))  
                if not os.path.exists(pathname):
                      os.makedirs(pathname)
                os.chdir(pathname)
                d = image_train[i]
                d.save("{0}.jpg".format(str(cnt)))
            if((label_train[i])==(label_train[i-1])):
                  n = label_train[i]
                  #print('cat', n)
                  d = image_train[i]
                  d.save("{0}.jpg".format(str(cnt)))
                
        cnt +=1

In [ ]:
len_cat = len(test_im)
#
cnt = 0
flag=0
for i in range(len_cat):
        if i == 0:
            n = test_lab[i]
            #print('cat', n)
            dir_ = 'C:\\Users\\saish\\Documents\\EE258\\test\\test'
            pathname = os.path.join(dir_, str(n))
            if not os.path.exists(pathname):
                  os.makedirs(pathname)
            os.chdir(pathname)
            d = test_im[i]
            d.save("{0}.jpg".format(str(cnt)))
        if i != 0:      
            if((test_lab[i])!=(test_lab[i-1])):
                n = test_lab[i]
                #print('cat', n)
                dir_ = 'C:\\Users\\saish\\Documents\\EE258\\test\\test'
                pathname = os.path.join(dir_, str(n))  
                if not os.path.exists(pathname):
                      os.makedirs(pathname)
                os.chdir(pathname)
                d = test_im[i]
                d.save("{0}.jpg".format(str(cnt)))
            if((test_lab[i])==(test_lab[i-1])):
                  n = test_lab[i]
                  #print('cat', n)
                  d = test_im[i]
                  d.save("{0}.jpg".format(str(cnt)))
                
        cnt +=1

# KERAS VGG16

In [ ]:
img_width, img_height = 180, 180
top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'C:\\Users\\saish\\Documents\\EE258\\test\\train'
validation_data_dir = 'C:\\Users\\saish\\Documents\\EE258\\test\\test'
nb_train_samples = len_train
nb_validation_samples = len_test
epochs = 50
batch_size = 1

In [ ]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('C:\\Users\\saish\\Documents\\EE258\\bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('C:\\Users\\saish\\Documents\\EE258\\bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)

In [ ]:
train_labels = np.array([0] * 1000 + [1] * 1000)
a = np.asarray(label_train)
a.shape

In [ ]:
save_bottlebeck_features()

# Freezing model

In [ ]:
#def train_top_model():
train_data = np.load(open('C:\\Users\\saish\\Documents\\EE258\\vgg16\\bottleneck_features_train.npy', 'rb'))
train_labels = np.array(label_train)


validation_data = np.load(open('C:\\Users\\saish\\Documents\\EE258\\vgg16\\bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array(test_lab)
    

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(len(rev_labels), activation='softmax'))

model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data,train_labels,validation_split=0.2,epochs=5,batch_size=1)
    
model.save_weights(top_model_weights_path)


# Experimenting with parameters

In [10]:
#def train_top_model():
train_data = np.load(open('C:\\Users\\saish\\Documents\\EE258\\vgg16\\bottleneck_features_train.npy', 'rb'))
train_labels = np.array(label_train)


validation_data = np.load(open('C:\\Users\\saish\\Documents\\EE258\\vgg16\\bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array(test_lab)
    

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(rev_labels), activation='softmax'))

model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#model.fit(train_data,train_labels,validation_split=0.2,epochs=5,batch_size=10)
    
#model.save_weights(top_model_weights_path)


In [11]:
model.fit(train_data,train_labels,validation_split=0.2,epochs=2,batch_size=10)

Train on 7984 samples, validate on 1996 samples
Epoch 1/2
7984/7984 [==============================] - 91s 11ms/step - loss: 4.8668 - acc: 0.3082 - val_loss: 4.1996 - val_acc: 0.3487
Epoch 2/2
7984/7984 [==============================] - 89s 11ms/step - loss: 4.7842 - acc: 0.3265 - val_loss: 4.2514 - val_acc: 0.3487


In [15]:
train_data = np.load(open('C:\\Users\\saish\\Documents\\EE258\\vgg16\\bottleneck_features_train.npy', 'rb'))
train_labels = np.array(label_train)


validation_data = np.load(open('C:\\Users\\saish\\Documents\\EE258\\vgg16\\bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array(test_lab)
    

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(len(rev_labels), activation='softmax'))

model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data,train_labels,validation_split=0.2,epochs=2,batch_size=10)
    

Train on 7984 samples, validate on 1996 samples
Epoch 1/2
7984/7984 [==============================] - 170s 21ms/step - loss: 5.1474 - acc: 0.3161 - val_loss: 4.1484 - val_acc: 0.3487
Epoch 2/2
7984/7984 [==============================] - 168s 21ms/step - loss: 4.3707 - acc: 0.3254 - val_loss: 4.4067 - val_acc: 0.3487


In [16]:
train_data = np.load(open('C:\\Users\\saish\\Documents\\EE258\\vgg16\\bottleneck_features_train.npy', 'rb'))
train_labels = np.array(label_train)


validation_data = np.load(open('C:\\Users\\saish\\Documents\\EE258\\vgg16\\bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array(test_lab)
    

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(len(rev_labels), activation='softmax'))

model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data,train_labels,validation_split=0.2,epochs=2,batch_size=10)

Train on 7984 samples, validate on 1996 samples
Epoch 1/2
7984/7984 [==============================] - 343s 43ms/step - loss: 10.8352 - acc: 0.3270 - val_loss: 10.4979 - val_acc: 0.3487
Epoch 2/2
7984/7984 [==============================] - 337s 42ms/step - loss: 10.8350 - acc: 0.3278 - val_loss: 10.4979 - val_acc: 0.3487


In [17]:
train_data = np.load(open('C:\\Users\\saish\\Documents\\EE258\\vgg16\\bottleneck_features_train.npy', 'rb'))
train_labels = np.array(label_train)


validation_data = np.load(open('C:\\Users\\saish\\Documents\\EE258\\vgg16\\bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array(test_lab)
    

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(len(rev_labels), activation='softmax'))

sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer='sgd',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data,train_labels,validation_split=0.2,epochs=2,batch_size=10)

Train on 7984 samples, validate on 1996 samples
Epoch 1/2
7984/7984 [==============================] - 223s 28ms/step - loss: 4.3528 - acc: 0.2885 - val_loss: 4.9082 - val_acc: 0.1127
Epoch 2/2
7984/7984 [==============================] - 225s 28ms/step - loss: 4.1736 - acc: 0.3041 - val_loss: 4.7920 - val_acc: 0.1167


In [19]:
from keras import regularizers
train_data = np.load(open('C:\\Users\\saish\\Documents\\EE258\\vgg16\\bottleneck_features_train.npy', 'rb'))
train_labels = np.array(label_train)


validation_data = np.load(open('C:\\Users\\saish\\Documents\\EE258\\vgg16\\bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array(test_lab)
    

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.1))
model.add(Dense(len(rev_labels), activation='softmax'))

sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer='sgd',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data,train_labels,validation_split=0.2,epochs=2,batch_size=10)

Train on 7984 samples, validate on 1996 samples
Epoch 1/2
7984/7984 [==============================] - 339s 42ms/step - loss: 20.5967 - acc: 0.2913 - val_loss: 17.9741 - val_acc: 0.3487
Epoch 2/2
7984/7984 [==============================] - 304s 38ms/step - loss: 15.9914 - acc: 0.3034 - val_loss: 14.1849 - val_acc: 0.1132
